In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**les importation nécessaire**

In [ ]:
import tensorflow.keras as K
import matplotlib.pyplot as plt

**Cette méthode faire le prétraitement pour former un modèle**

In [ ]:
def preprocess_data(X, Y):
        X = X.astype('float32')
        Y = Y.astype('float32')
        x_pr = K.applications.vgg16.preprocess_input(X)
        y_pr = K.utils.to_categorical(Y, 10)
        return(x_pr, y_pr)

**chargement des données et utilisation du prétraitement pour l'apprentissage et la validation de l'ensemble de données**

In [ ]:
(Xt, Yt), (X, Y) = K.datasets.cifar10.load_data()
X_train, Y_train = preprocess_data(Xt, Yt)
X_test, Y_test= preprocess_data(X, Y)

**Création du modéle VGG16**

In [ ]:
model1 = K.applications.vgg16.VGG16(include_top=False,
                                            weights='imagenet',
                                            pooling='avg'
                                            )

model= K.Sequential()
model.add(K.layers.UpSampling2D())
model.add(model1)
model.add(K.layers.Flatten())
model.add(K.layers.Dense(512, activation=('relu'))) 
model.add(K.layers.Dropout(0.2))
model.add(K.layers.Dense(256, activation=('relu')))
model.add(K.layers.Dropout(0.2))
model.add(K.layers.Dense(10, activation=('softmax')))

**Cette méthode sert à crée Alpha**

In [ ]:
def decay(epoch):
    return 0.001 / (1 + 1 * 30)

**Modèle de compilation**

In [ ]:
callback = []
callback += [K.callbacks.LearningRateScheduler(decay, verbose=1)]
callback += [K.callbacks.ModelCheckpoint('cifar10.h5',save_best_only=True,mode='min' )]
model.compile(optimizer='adam', loss='categorical_crossentropy',
                      metrics=['accuracy'])

****Entraînement****

In [ ]:
m_entrain=model.fit(x=X_train, y=Y_train,
              batch_size=128,
              validation_data=(X_test, Y_test),
              epochs=25, shuffle=True,
              callbacks=callback,
              verbose=1
              )

**Historique de modéle**

In [ ]:
model.summary()

**Évaluation**

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('test loss est',score[0])
print('test accuracy est',score[1])

**Visualisation des résultat dans un graphe**

In [ ]:
def figure(h, nb_epoch):
        acc, loss, val_acc, val_loss = h.history['accuracy'], h.history['loss'], h.history['val_accuracy'], h.history['val_loss']
        plt.figure(figsize=(15, 5))
        plt.subplot(121)
        plt.plot(range(nb_epoch), acc, label='Train')
        plt.plot(range(nb_epoch), val_acc, label='Test')
        plt.title('Accuracy (' + str(nb_epoch) + ' Epochs )', size=15)
        plt.legend()
        plt.grid(True)
        plt.subplot(122)
        plt.plot(range(nb_epoch), loss, label='Train')
        plt.plot(range(nb_epoch), val_loss, label='Test')
        plt.title('Loss (' + str(nb_epoch) + ' Epochs )', size=15)
        plt.legend()
        plt.grid(True)
        plt.show()

In [ ]:
figure(m_entrain, 25)